In [1]:
import Base.copy
using Distributions
include("Mondrian_Forest_Classifier.jl"); 
include("Mondrian_extention.jl")

expand!

In [2]:
function 
    FakedataClassif(n_obs,d)
    x=randn((n_obs,d))
    y=(sum(x*randn(d),2) .> mean(sum(x*randn(d),2)))
    y=y+1
    y=Int.(y)
    return x,y
end

FakedataClassif (generic function with 2 methods)

In [3]:
data = FakedataClassif(50000,100)
d_new=FakedataClassif(1000,100)

([0.152564 -1.14849 … -0.849731 0.603876; 0.0502868 0.0862814 … -1.13152 0.647914; … ; 1.34466 -0.366733 … 0.263148 -2.95667; -0.316246 -0.971352 … 0.992023 -0.581751], [2, 2, 2, 1, 2, 1, 1, 2, 2, 2  …  2, 2, 2, 1, 1, 1, 2, 2, 2, 1])

In [4]:
T = Mondrian_Tree()

Mondrian_Tree(Nullable{Mondrian_Node}(), Mondrian_Node[])

In [5]:
T_online = Sample_Mondrian_Tree!(T,1e9,data[1],data[2]);  #compute base tree

In [6]:
T_online=expand!(T_online,d_new[1],d_new[2],1e9);

In [7]:
All_data = zeros(length(data[2])+length(d_new[2]),length(data[1][1,:]))
All_data[1:length(data[2]),:]=data[1]
All_data[length(data[2])+1:length(data[2])+length(d_new[2]),:]=d_new[1]

all_labels = Array{Int64}(length(data[2])+length(d_new[2]),1)
all_labels[1:length(data[2]),:]=data[2]
all_labels[length(data[2])+1:length(data[2])+length(d_new[2]),:]=d_new[2];

In [8]:
T_full = Sample_Mondrian_Tree!(T,1e9,All_data,all_labels);  #compute full tree of the whole dataset

### Compare prediction accuracy

In [9]:
compute_predictive_posterior_distribution!(T_full,10*size(All_data,2))

In [10]:
pred = []
    for i in 1:size(All_data,1)
        p = predict!(T_full,All_data[i,:],10*size(All_data,2))
        if p[1] > p[2]
            push!(pred,1)
        else
            push!(pred,2)
        end
    end


In [11]:
counter = 0
for i=1:size(pred)[1]
    if pred[i]==all_labels[i]
        counter = counter+1
    end
end
counter = counter /size(pred)[1]

0.5024705882352941

In [12]:
compute_predictive_posterior_distribution!(T_online,10*size(All_data,2))

In [13]:
 pred = []
        for i in 1:size(All_data,1)
            p = predict!(T_online,All_data[i,:],10*size(All_data,2))
            if p[1] > p[2]
                push!(pred,1)
            else
                push!(pred,2)
            end
        end

In [14]:
counter = 0
for i=1:size(pred)[1]
    if pred[i]==all_labels[i]
        counter = counter+1
    end
end
counter = counter /size(pred)[1]

0.5024705882352941

## Add Docs

In [22]:
# #Utils for extending Mondrian Trees 

# """
# `function Extended_dimension(Θ::Axis_Aligned_Box,X::Array{Float64})`

# This function samples the rate ∑ᵢ₌₁ᴰmax(lⱼ-x,0)+max(x-uⱼ,0) with which we draw the parameter E from the exponential distribution in the function Extend_Mondrian_Block!. 

# `Input`:  Axis Aligned Box Θ (Abstract type Axis_Aligned_Box), One dimensional array of features X (Array of Float64)

# `Output`: Sampling rate s

# `Files needed to run this function`: "Mondrian_extention.jl"

# This function is one of the utility functions to run "Extend_Mondrian_Block!".


# """

# function Extended_dimension(Θ::Axis_Aligned_Box,X::Array{Float64}) 
#     # rate = ∑ᵢ₌₁ᴰmax(lⱼ-x,0)+max(x-uⱼ,0)
    
#     s = 0
#     for i in 1:Θ.D
#         s += max(Θ.Intervals[i,1]-X[i],0)+max(X[i]-Θ.Intervals[i,2],0)  #compute the rate
#     end
#     return s
# end

# """
# `function sample_extended_split_dimension(Θ::Axis_Aligned_Box,X::Array{Float64})`

# This function samples the split dimension and direction associated to the new node j_wave in the function "Extend_Mondrian_Block!"

# `Input`: Axis Aligned Box Θ (abstract type Axis_Aligned_Box), One dimensional array of features X (Array of Float64)

# `Output`: Split dimension d, split direction s

# `Files needed to run this function`: "Mondrian_extention.jl"

# This function is one of the utility functions to run "Extend_Mondrian_Block!".
# """

# function sample_extended_split_dimension(Θ::Axis_Aligned_Box,X::Array{Float64}) 
#     p_k = zeros(Θ.D)
#     for i in 1:Θ.D
#         p_k[i] = max(Θ.Intervals[i,1]-X[i],0)+max(X[i]-Θ.Intervals[i,2],0)  #compute max(lⱼ-x,0)+max(x-uⱼ,0)
#     end
#     p_k = p_k ./ Extended_dimension(Θ,X)                                #create the probability array
#     d = rand(Categorical(p_k))                                          # sample from it (d with probability proportional to max(lⱼ-x,0)+max(x-uⱼ,0) )

#     if X[d] > Θ.Intervals[d,2]  # choice of interval dependent on where the new datapoint is                                    
#     x = rand(Uniform(Θ.Intervals[d,2][1],X[d]))  #sample the split loc. uniformly from that interval
#     else
#     x = rand(Uniform(X[d],Θ.Intervals[d,1][1])) #sample the split loc. uniformly from that interval
#     end
#     return d,x   #return split dim., split loc. 
# end

# """
# `function update_intervals(Θ::Axis_Aligned_Box,X::Array{Float64})`

# This function computes the boxes associated to the new node j_wave. 

# `Input`: Axis Aligned Box Θ (abstract type Axis_Aligned_Box), One dimensional array of features X (Array of Float64)

# `Output`: Axis Aligned Box Θ (updated Intervals)

# `Files needed to run this function`: "Mondrian_extention.jl"

# This function is one of the utility functions to run "Extend_Mondrian_Block!".

# """
# function update_intervals(Θ::Axis_Aligned_Box,X::Array{Float64})
#     Intervals = Θ.Intervals                         # extract the intervals from Θ
#    for i=1:length(Θ.Intervals[:,1])                 # update the intervals successively 
#        l=min(Intervals[i,1],X[i])
#        u=max(Intervals[i,2],X[i])
#        Intervals[i,:]=[l,u]
#     end
#     Θ = Axis_Aligned_Box(Intervals,Θ.D)             # return updated intervals
#     return Θ
# end

update_intervals

In [23]:
?update_intervals

search:



`function update_intervals(Θ::Axis_Aligned_Box,X::Array{Float64})`

This function computes the boxes associated to the new node j_wave. 

`Input`: Axis Aligned Box Θ (abstract type Axis_Aligned_Box), One dimensional array of features X (Array of Float64)

`Output`: Axis Aligned Box Θ (updated Intervals)

`Files needed to run this function`: "Mondrian_extention.jl"

This function is one of the utility functions to run "Extend_Mondrian_Block!".


## To Do:

Clean up utils files, test on more classes, implement paused mondrians 